In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.model_selection as sk
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K

## 1. Load & Prepare Data

In [9]:
# merge features and label
#df= pd.read_csv('25K_tracks_features_and_labels_all.csv')
df= pd.read_csv('../../../data/processed/ohe_25K_tracks_features_and_labels_all.csv')
df

,Unnamed: 0,track_id,track_genre_top,track_title,artist_name,set_split,set_subset,tempo,rms_harmonic_mean,rms_harmonic_var,...,key_C,key_C#,key_D,key_E,key_Eb,key_F,key_F#,key_G,scale_major,scale_minor
0,0,2,Hip-Hop,Food,AWOL,training,small,161.499023,0.060801,0.000606,...,1,0,0,0,0,0,0,0,0,1
1,1,3,Hip-Hop,Electric Ave,AWOL,training,medium,86.132812,0.068484,0.001239,...,0,0,0,0,0,0,0,1,0,1
2,2,5,Hip-Hop,This World,AWOL,training,small,99.384014,0.079553,0.002230,...,0,1,0,0,0,0,0,0,0,1
3,3,10,Pop,Freeway,Kurt Vile,training,small,112.347147,0.135624,0.000600,...,0,0,0,0,0,0,1,0,1,0
4,4,134,Hip-Hop,Street Music,AWOL,training,medium,135.999178,0.088261,0.001123,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24767,24995,155297,Instrumental,Nebula Reborn,Alex Mason/BlackSunAeon Music,training,medium,123.046875,0.038447,0.000094,...,0,0,1,0,0,0,0,0,1,0
24768,24996,155298,Folk,An Idiot Abroad,Greg Atkinson,training,medium,129.199219,0.139233,0.003167,...,0,0,0,0,0,0,0,1,1,0
24769,24997,155306,Folk,Tiny Man,Greg Atkinson,training,medium,103.359375,0.116513,0.003872,...,0,0,0,0,0,0,0,0,1,0
24770,24998,155307,Experimental,Kolka,AWOTT,training,medium,117.453835,0.085435,0.001713,...,1,0,0,0,0,0,0,0,0,1


In [10]:
df_train = df.copy()
df_train['target_encoded'] = LabelEncoder().fit_transform(df_train['track_genre_top'])

In [11]:
genre_target_map = df_train[['target_encoded' ,'track_genre_top']]
genre_target_map = genre_target_map.drop_duplicates(keep='last')
genre_target_map.to_csv('label_target_mapping.csv')

In [12]:
#convert categorical variables/target class to code
df_train = df.copy()
# df_train['key'] = LabelEncoder().fit_transform(df_train['key'])
# df_train['scale'] = LabelEncoder().fit_transform(df_train['scale'])
df_train['track_genre_top'] = LabelEncoder().fit_transform(df_train['track_genre_top'])

feature_cols = list(df_train.columns[7:])
print(f'There are {len(feature_cols)} features')
print(feature_cols)
target_col = 'track_genre_top'
classes = df[target_col].unique()
num_class = len(classes)
print(f'There are {num_class} classes, they are:', classes)

There are 88 features
['tempo', 'rms_harmonic_mean', 'rms_harmonic_var', 'rms_percussive_mean', 'rms_percussive_var', 'spectral_centroid_mean', 'spectral_centroid_var', 'spectral_rolloff_mean', 'spectral_rolloff_var', 'zero_crossings', 'chroma_stft_0_mean', 'chroma_stft_1_mean', 'chroma_stft_2_mean', 'chroma_stft_3_mean', 'chroma_stft_4_mean', 'chroma_stft_5_mean', 'chroma_stft_6_mean', 'chroma_stft_7_mean', 'chroma_stft_8_mean', 'chroma_stft_9_mean', 'chroma_stft_10_mean', 'chroma_stft_11_mean', 'chroma_stft_0_var', 'chroma_stft_1_var', 'chroma_stft_2_var', 'chroma_stft_3_var', 'chroma_stft_4_var', 'chroma_stft_5_var', 'chroma_stft_6_var', 'chroma_stft_7_var', 'chroma_stft_8_var', 'chroma_stft_9_var', 'chroma_stft_10_var', 'chroma_stft_11_var', 'MFCC_1_mean', 'MFCC_2_mean', 'MFCC_3_mean', 'MFCC_4_mean', 'MFCC_5_mean', 'MFCC_6_mean', 'MFCC_7_mean', 'MFCC_8_mean', 'MFCC_9_mean', 'MFCC_10_mean', 'MFCC_11_mean', 'MFCC_12_mean', 'MFCC_13_mean', 'MFCC_14_mean', 'MFCC_15_mean', 'MFCC_16_mean

In [13]:
df[target_col].value_counts()

Rock                   7098
Electronic             6311
Experimental           2250
Hip-Hop                2197
Folk                   1518
Instrumental           1349
Pop                    1186
International          1018
Classical               619
Old-Time / Historic     510
Jazz                    384
Country                 178
Soul-RnB                154
Name: track_genre_top, dtype: int64

In [15]:
#Define train/test/validation & apply scaling
df_training = df_train.loc[df_train['set_split']=='training'].dropna()
df_testing = df_train.loc[df_train['set_split']=='test'].dropna()
df_validation = df_train.loc[df_train['set_split']=='validation'].dropna()

scaler = StandardScaler()

X_train = scaler.fit_transform(df_training[feature_cols])
y_train =  df_training[target_col].values
X_test = scaler.fit_transform(df_testing[feature_cols])
y_test = df_testing[target_col].values
X_val = scaler.fit_transform(df_validation[feature_cols])
y_val = df_validation[target_col].values
print('Training data:',X_train.shape)
print('Testing data:',X_test.shape)
print('Validation data:',X_val.shape)

Training data: (19744, 88)
Testing data: (2546, 88)
Validation data: (2482, 88)


In [16]:
#Oversample minority classes
##In the end we did not use oversampled because it performed worse than un-sampled data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print('Oversampled X_train:', X_train_resampled.shape)

Oversampled X_train: (73801, 88)


## 2. Build Models & Tune Hyperparameters
#### *You can skip this part and go straight to running the final model

In [ ]:


# Define hyperparameters to tune
learning_rates = [0.0001, 0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
hidden_layer_sizes = [[128,64,32], [256,128,64,32], [512,256,128,64,32]]
dropout_rates = [0.2, 0.5]


best_accuracy = 0
best_hyperparameters = None

# Grid search over hyperparameters
for lr in learning_rates:
    for batch_size in batch_sizes:
          for dropout_rate in dropout_rates:
              K.clear_session()  # Clear previous model
              model = Sequential()

              for hidden_size in hidden_layer_sizes:
                  for size in hidden_size:
                      model.add(Dense(size, activation='relu'))
                      model.add(Dropout(dropout_rate))
              model.add(Dense(num_class, activation='softmax'))

              optimizer = Adam(learning_rate=lr)
              model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

              # Early stopping to prevent overfitting
              early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

              history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                                    epochs=20, batch_size=batch_size, callbacks=[early_stopping])

              # Evaluate model on validation set
              _, accuracy = model.evaluate(X_val, y_val, verbose=0)

              # Check if this set of hyperparameters improved accuracy
              if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_hyperparameters = {'lr': lr, 'batch_size': batch_size, 'hidden_layer_size': hidden_size, 'dropout_rate': dropout_rate}
                    best_model = model
# Print the best hyperparameters
print("Best Hyperparameters:", best_hyperparameters)

In [9]:
best_hyperparameters

{'lr': 0.001,
 'batch_size': 128,
 'hidden_layer_size': [512, 256, 128, 64, 32],
 'dropout_rate': 0.2}

In [10]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, batch_size=128)
print("The test loss is :",test_loss)
print("\nThe test Accuracy is :",test_accuracy)

20/20 [==============================] - 0s 2ms/step - loss: 1.4869 - accuracy: 0.5416
The test loss is : 1.4869064092636108

The test Accuracy is : 0.5416339635848999


## 3. Retrain final model using the best model architecture & hyperparameters

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

model = keras.models.Sequential([
    keras.layers.Dense(512, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(256,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(num_class, activation="softmax"),

])
print(model.summary())
model.compile(optimizer=Adam(lr=0.001),
                  loss='sparse_categorical_crossentropy',
                   metrics='accuracy')

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30,
                     batch_size=128)

test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=128)
print("The test loss is :",test_loss)
print("\nThe test Accuracy is :",test_accuracy)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               45568     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8

None
Epoch 1/30
155/155 [==============================] - 1s 4ms/step - loss: 1.7292 - accuracy: 0.4545 - val_loss: 1.3694 - val_accuracy: 0.5999
Epoch 2/30
155/155 [==============================] - 1s 4ms/step - loss: 1.4722 - accuracy: 0.5423 - val_loss: 1.3016 - val_accuracy: 0.6072
Epoch 3/30
155/155 [==============================] - 1s 4ms/step - loss: 1.3936 - accuracy: 0.5727 - val_loss: 1.2816 - val_accuracy: 0.6096
Epoch 4/30
155/155 [==============================] - 1s 4ms/step - loss: 1.3337 - accuracy: 0.5918 - val_loss: 1.2480 - val_accuracy: 0.6217
Epoch 5/30
155/155 [==============================] - 1s 4ms/step - loss: 1.2923 - accuracy: 0.6056 - val_loss: 1.2343 - val_accuracy: 0.6221
Epoch 6/30
155/155 [==============================] - 1s 4ms/step - loss: 1.2489 - accuracy: 0.6169 - val_loss: 1.2384 - val_accuracy: 0.6205
Epoch 7/30
155/155 [==============================] - 1s 4ms/step - loss: 1.2231 - accuracy: 0.6247 - val_loss: 1.2202 - val_accuracy: 0.6193
E

In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Use argmax for categorical predictions


# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

# Print accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy:", accuracy)

# Print precision
precision = precision_score(y_test, y_pred_classes, average='weighted')
print("Precision:", precision)

# Print recall
recall = recall_score(y_test, y_pred_classes, average='weighted')
print("Recall:", recall)

# Print F1 score
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("F1 Score:", f1)

80/80 [==============================] - 0s 723us/step
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.69      0.74        62
           1       0.00      0.00      0.00        18
           2       0.61      0.70      0.65       632
           3       0.39      0.32      0.35       225
           4       0.25      0.26      0.25       152
           5       0.62      0.68      0.65       220
           6       0.35      0.16      0.22       174
           7       0.40      0.33      0.36       102
           8       0.47      0.38      0.42        39
           9       0.91      0.96      0.93        51
          10       0.14      0.09      0.11       119
          11       0.66      0.79      0.72       710
          12       0.00      0.00      0.00        42

    accuracy                           0.57      2546
   macro avg       0.43      0.41      0.42      2546
weighted avg       0.53      0.57      0.54      2546

C

/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

In [19]:
confusion_matrix(y_test, y_pred_classes)

array([[ 43,   0,   1,   4,   1,   1,   9,   1,   1,   0,   0,   1,   0],
       [  0,   0,   4,   0,   0,   4,   0,   0,   0,   0,   1,   9,   0],
       [  4,   1, 443,  35,   9,  41,  14,  15,   1,   0,  11,  58,   0],
       [  1,   0,  58,  72,  16,   5,  11,   5,   3,   0,   5,  49,   0],
       [  0,   2,  16,  14,  39,   0,   5,   7,   2,   4,  15,  48,   0],
       [  0,   0,  46,   4,   1, 149,   1,   6,   2,   0,   2,   9,   0],
       [  2,   0,  31,  23,  41,   1,  27,   1,   1,   0,  10,  37,   0],
       [  2,   0,  12,   1,  21,   5,   1,  34,   0,   0,   5,  21,   0],
       [  2,   0,   4,   2,   5,   0,   4,   2,  15,   0,   0,   5,   0],
       [  0,   0,   0,   2,   0,   0,   0,   0,   0,  49,   0,   0,   0],
       [  0,   1,  45,   6,   9,  12,   0,   5,   2,   0,  11,  28,   0],
       [  1,   2,  50,  22,  17,  20,   5,   8,   4,   1,  16, 564,   0],
       [  0,   0,  11,   0,   0,   4,   0,   1,   1,   0,   0,  25,   0]])

In [20]:
model.save('../../../models/final_models/cnn_genre_prediction.h5')

/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
type(X_train)

numpy.ndarray

In [22]:
# to get feature importances

import shap

background = X_train[np.random.choice(X_train.shape[0], 10000, replace=False)]

# Create a SHAP Explainer
explainer = shap.Explainer(model, background)

# Select a sample (or multiple samples) for explanation
sample = X_test[np.random.choice(X_test.shape[0], 1000, replace=False)]

# Calculate SHAP values
shap_values = explainer(sample)


PermutationExplainer explainer: 1001it [01:26, 10.48it/s]                          


In [88]:
feature_importances_per_class = {}

# Iterate over each class (label)
for label_index in range(shap_values.shape[2]):
    # Calculate mean absolute SHAP values for each feature for this label
    mean_abs_shap_per_feature = np.abs(shap_values.values[:, :, label_index]).mean(axis=0)
    
    # Assuming 'feature_cols' is a list of your feature names
    feature_importances_per_class[f'Label_{label_index}'] = dict(zip(feature_cols, mean_abs_shap_per_feature))

# Convert the dictionary to a DataFrame for easier handling
df_feature_importances = pd.DataFrame(feature_importances_per_class)



In [99]:
cnn_fi_df = pd.DataFrame(df_feature_importances.T.mean()).T

In [101]:
cnn_fi_df.to_csv('../../../data/resources/cnn_feature_importances.csv')